## Superwęzeł - supernode

Superwęzeł (ang. supernode) to węzeł w grafie, który ma bardzo dużą liczbę relacji (np. setki tysięcy lub miliony).

Przykłady to:
- węzeł :User reprezentujący celebrytę z milionem obserwujących,
- węzeł :Tag użyty przez milion postów,
- węzeł :Product kupiony przez setki tysięcy klientów.

### Dlaczego należy ich unikać?
- Wydajność: zapytania zaczynające się od superwęzła mogą być bardzo wolne – Neo4j musi przeszukać ogromny zestaw relacji, zanim przefiltruje te istotne.
- Przeciążenie pamięci: graf musi załadować dużą liczbę relacji do RAM-u, co obciąża system.
- Słaba selektywność zapytań: indeksy nie pomagają, gdy i tak trzeba przetworzyć tysiące relacji z jednego węzła.

### Jak unikać superwęzłów?
- Segmentuj dane – np. :Post → :Tag_2025, :Customer_US, itp.
- Dodawaj właściwości do relacji (np. :LIKED {timestamp}) i filtruj po nich.
- Ograniczaj kierunek przeszukiwania w zapytaniach, np. [:FRIENDS_WITH]-> zamiast [:FRIENDS_WITH*].
- Partycjonowanie tagów:
    - Zamiast jednego (:Tag {name: "AI"}) → użyj np. (:Tag {name: "AI", year: 2024})
- Zastąpienie relacji właściwością (jeśli nie analizujesz relacji):
    - :Post {tags: ["AI", "ML"]} – wtedy szukasz WHERE "AI" IN p.tags, ale tracisz semantykę grafu.

Laboratorium: Superwęzeł – problem i optymalizacja
Cel ćwiczenia:
- Zobaczyć wpływ superwęzła na wydajność zapytań.
- Nauczyć się optymalizować strukturę grafu: przez filtrowanie, partycjonowanie, projektowanie pod zapytania.

In [3]:
from neo4j import GraphDatabase
import pandas as pd

In [4]:
# Połączenie z Neo4j
URI = "bolt://neo4j_nosql_lab:7687"
USERNAME = "neo4j"
PASSWORD = "test1234"  # upewnij się, że to pasuje do graph-docker-compose.yml

In [ ]:
q_list = [
    # // Tworzenie indeksu
    "DROP INDEX idx_post_created_at IF EXISTS;"
    , "CREATE INDEX idx_post_created_at FOR (p:Post) ON (p.created_at);"

    # // Superwęzeł: jeden tag
    , "CREATE (:Tag {name: 'AI'});"

    # // Tworzenie 10k postów z relacją do tagu
    , """UNWIND range(1, 10000) AS i
    CREATE (p:Post {title: 'Post #' + i, created_at: date('2025-01-01')})
    WITH p
    MATCH (t:Tag {name: 'AI'})
    CREATE (p)-[:HAS_TAG]->(t);"""
]

In [14]:
def run_cypher(query: str, driver):
    with driver.session() as session:
        result = session.run(query)
        records = list(result)
        if not records:
            print("🔍 Brak wyników.")
            return
        # Przekształcenie do DataFrame
        df = pd.DataFrame([r.data() for r in records])
        return df

print("Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.")

def qexec(query: str):
    driver = GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))
    
    df = run_cypher(query, driver)
    
    driver.close()
    
    return df

Połączono z Neo4j. Użyj funkcji `run_cypher('TWOJE_ZAPYTANIE')` aby wykonać zapytanie.


In [ ]:
for q in q_list:
    print(q)
    qexec(q)

### Zadanie 1: Zapytanie bez filtru (klasyczny superwęzeł)

In [ ]:
q = """
MATCH (t:Tag {name: 'AI'})<-[:HAS_TAG]-(p:Post)
RETURN p.title
LIMIT 5
"""
qexec(q)

### Zadanie 2: Optymalizacja – filtr po dacie

In [ ]:
q = """
MATCH (t:Tag {name: 'AI'})<-[:HAS_TAG]-(p:Post)
WHERE p.created_at > date('2025-01-01')
RETURN p.title
LIMIT 5
"""
qexec(q)

### Zadanie 3: Partycjonowanie tagów po roku

In [ ]:
q = """
// Tworzenie tagu AI_2025
CREATE (:Tag {name: 'AI_2025'});

// Tworzenie 10k postów przypisanych tylko do partycji
UNWIND range(1, 10000) AS i
CREATE (p:Post {title: 'Post_2025_' + i, created_at: date('2025-01-01')})
WITH p
MATCH (t:Tag {name: 'AI_2025'})
CREATE (p)-[:HAS_TAG]->(t);
"""
qexec(q)

In [ ]:
query = """
// Zapytanie po partycjonowanym tagu
MATCH (t:Tag {name: 'AI_2025'})<-[:HAS_TAG]-(p:Post)
RETURN p.title
LIMIT 5
"""
qexec(query)

### Zadanie 4: Porównanie czasu zapytania

In [ ]:
query = """
PROFILE MATCH (...) RETURN ...
"""
qexec(query)

### Checklista: cecha jako węzeł czy właściwość?

| Kryterium                                                                 | Odpowiedź | Zalecenie     |
|---------------------------------------------------------------------------|-----------|----------------|
| Czy wiele węzłów może dzielić tę samą cechę?                              | Tak       | Węzeł          |
| Czy chcesz zadawać pytania w stylu: „Pokaż wszystkie węzły z tą cechą”?  | Tak       | Węzeł          |
| Czy cecha ma własne właściwości (np. `name`, `created_at`)?              | Tak       | Węzeł          |
| Czy cecha może mieć relacje z innymi węzłami?                             | Tak       | Węzeł          |
| Czy cecha występuje wielokrotnie u jednego węzła (lista, zbiór)?         | Tak       | Węzeł          |
| Czy chcesz analizować tę cechę jako część grafu (powiązania, agregacje)? | Tak       | Węzeł          |
| Czy cecha jest unikalna tylko dla tego jednego węzła?                    | Tak       | Właściwość     |
| Czy to prosty opis, bez relacji i analizy (np. wiek, status)?            | Tak       | Właściwość     |
| Czy cecha jest używana do filtrowania lub indeksowania (np. `username`)? | Tak       | Właściwość     |
| Czy zależy Ci na maksymalnej wydajności zapytań po tej cesze?            | Tak       | Właściwość     |
